In [17]:
%matplotlib inline
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda,GRU,Embedding,LSTM
from keras.optimizers import RMSprop
from keras import backend as K
import keras
import numpy as np
import pandas as pd
import seaborn as sns

In [4]:
input1=np.load('Essay grader/input.npy')
output1=np.load('Essay grader/output.npy')

#  Train,Validation, Test  Split

In [5]:
train_input=input1[0:8000]
validate_input=input1[8000:8500]
test_input=input1[8500:]


In [6]:
train_output=output1[0:8000]
validate_output=output1[8000:8500]
test_output=output1[8500:]


# Word_Vectors

In [7]:
word_embedding=pd.read_csv('glove.6B/vocab2.csv')


In [8]:
word_embedding=word_embedding.drop(word_embedding.columns[0:1],axis=1)

In [9]:
word_embedding=word_embedding.as_matrix()

# Main_Model

## LSTM

In [10]:
model=Sequential()
model.add(Embedding(word_embedding.shape[0],word_embedding.shape[1],input_length=550,mask_zero=True,trainable=False,weights=[word_embedding]))
model.add(LSTM(output_dim=128,return_sequences=False,init='he_normal'))
model.add(Dropout(0.5))
model.add(Dense(13,activation='softmax'))

## Deep-LSTM 

In [27]:
model1=Sequential()
model1.add(Embedding(word_embedding.shape[0],word_embedding.shape[1],input_length=550,mask_zero=True,trainable=False,weights=[word_embedding]))
model1.add(LSTM(output_dim=128,return_sequences=True,init='he_normal'))
model1.add(LSTM(output_dim=128,return_sequences=True,init='he_normal'))
model1.add(LSTM(output_dim=128,return_sequences=False,init='he_normal'))
model1.add(Dropout(0.5))
model1.add(Dense(13,activation='softmax'))

In [21]:
model1.layers[1].input_shape

(None, 550, 100)

# Bidirection LSTM

In [14]:
model2=Sequential()
model2.add(Embedding(word_embedding.shape[0],word_embedding.shape[1],input_length=550,mask_zero=True,trainable=False,weights=[word_embedding]))
model2.add(LSTM(output_dim=128,return_sequences=False,go_backwards=True,init='he_normal'))
model2.add(Dropout(0.5))
model2.add(Dense(13,activation='softmax'))

In [15]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

# Checkpoints for saving model where val-acc improve

In [18]:
filepath='weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_weights_only=True, mode='max')
callbacks_list = [checkpoint]

# Training

In [60]:
model.fit(train_input,train_output,batch_size=128,nb_epoch=40,validation_data=[validate_input,validate_output])

Train on 8000 samples, validate on 500 samples
Epoch 1/40
8000/8000 [==============================] - 244s - loss: 1.0662 - categorical_accuracy: 0.5694 - val_loss: 1.0998 - val_categorical_accuracy: 0.5580

In [63]:
model.save('/home/nancy/Downloads/Essay grader/weights1.h5')

# Kappa Score for Evalution

In [19]:
import numpy as np

def quadratic_kappa(y, t, eps=1e-15):
      # Assuming y and t are one-hot encoded!
      num_scored_items = y.shape[0]
      num_ratings = y.shape[1] 
      ratings_mat = np.tile(np.arange(0, num_ratings)[:, None], 
                            reps=(1, num_ratings))
      ratings_squared = (ratings_mat - ratings_mat.T) ** 2
      weights = ratings_squared / (float(num_ratings) - 1) ** 2
	
      # We norm for consistency with other variations.
      y_norm = y / (eps + y.sum(axis=1)[:, None])
	
      # The histograms of the raters.
      hist_rater_a = y_norm.sum(axis=0)
      hist_rater_b = t.sum(axis=0)
	
      # The confusion matrix.
      conf_mat = np.dot(y_norm.T, t)
	
      # The nominator.
      nom = np.sum(weights * conf_mat)
      expected_probs = np.dot(hist_rater_a[:, None], 
                              hist_rater_b[None, :])
      # The denominator.
      denom = np.sum(weights * expected_probs / num_scored_items)
	
      return 1 - nom / denom

In [31]:
a=model.predict(test_input,batch_size=128)

In [33]:
quadratic_kappa(a,test_output)

0.82